In [1]:
import pandas as pd
import numpy as np

In [2]:
# Specify the columns you need here
cem_col = {"albums": [],
           "artists": [],
           "songs": []}
marlize_col = {"albums": ["_id", "id_artist", "genre", "publicationDate"],
               "artists": ["_id", "location"],
               "songs": ["id_album", "award"]}
joris_col = {"albums": ["_id", "id_artist"],
             "artists": ["_id", "type", "gender", "members"],
             "songs": ["id_album", "genre"]}
ppl_col = [cem_col, marlize_col, joris_col]

# get unique values of columns needed
albums_col = list(set(np.concatenate([d["albums"] for d in ppl_col])))
artists_col = list(set(np.concatenate([d["artists"] for d in ppl_col])))
songs_col = list(set(np.concatenate([d["songs"] for d in ppl_col])))

# assuming the data folders are in your local folder
albums_data = pd.read_csv("../data/wasabi_albums.csv", usecols=albums_col)
artists_data = pd.read_csv("../data/wasabi_artists.csv", usecols=artists_col)
songs_data = pd.read_csv("../data/wasabi_songs.csv", sep="\t", usecols=songs_col)

/home/joris/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
display("albums", albums_data.head())
display("artists", artists_data.head())
display("songs", songs_data.head())

'albums'

,_id,genre,id_artist,publicationDate
0,ObjectId(5714debb25ac0d8aee34d59a),Alternative Rock,ObjectId(56d7e91b6b60c09814f93e4a),1997
1,ObjectId(5714debb25ac0d8aee34d59b),Alternative Rock,ObjectId(56d7e91b6b60c09814f93e4a),1999
2,ObjectId(5714debb25ac0d8aee34d59c),Alternative Rock,ObjectId(56d7e91b6b60c09814f93e4a),2002
3,ObjectId(5714debb25ac0d8aee34d59d),Alternative Rock,ObjectId(56d7e91b6b60c09814f93e4a),2005
4,ObjectId(5714debb25ac0d8aee34d59e),NaN,ObjectId(56d7e91b6b60c09814f93e4a),NaN


'artists'

,_id,gender,location,members,type
0,ObjectId(56d7e91b6b60c09814f93e4a),NaN,"{""id_city_musicbrainz"":""6e2d2d30-dbc9-4d27-99f...","[{""id_member_musicbrainz"":""3ec05e94-bf6e-439f-...",Group
1,ObjectId(56d7e91c6b60c09814f93e4c),NaN,"{""id_city_musicbrainz"":"""",""country"":""Japan"",""c...","[{""id_member_musicbrainz"":""82bd3da4-7085-40b8-...",Group
2,ObjectId(56d7e91d6b60c09814f93e4e),NaN,"{""id_city_musicbrainz"":"""",""country"":""Netherlan...","[{""id_member_musicbrainz"":""2931cbb9-56a0-4a96-...",Group
3,ObjectId(56d7e91e6b60c09814f93e50),NaN,"{""id_city_musicbrainz"":"""",""country"":"""",""city"":""""}",[],NaN
4,ObjectId(56d7e91e6b60c09814f93e52),NaN,"{""id_city_musicbrainz"":"""",""country"":""United St...","[{""id_member_musicbrainz"":""2f647f4c-6272-4ad9-...",Group


'songs'

,award,genre,id_album
0,NaN,NaN,ObjectId(5714debb25ac0d8aee34d59a)
1,NaN,NaN,ObjectId(5714debb25ac0d8aee34d59a)
2,NaN,NaN,ObjectId(5714debb25ac0d8aee34d59a)
3,NaN,NaN,ObjectId(5714debb25ac0d8aee34d59a)
4,NaN,NaN,ObjectId(5714debb25ac0d8aee34d59a)


In [4]:
results = pd.DataFrame(columns=["source","target","value"])

In [5]:
artists_data["type"].value_counts()

Group        29806
Person       24264
Choir           44
Orchestra       30
Character       25
Other           21
Name: type, dtype: int64

In [6]:
artists_data["gender"].value_counts()

Male      14309
Female     5881
Other         8
Name: gender, dtype: int64

In [7]:
df = pd.melt(artists_data, ["type"], ["gender"])

In [9]:
df.value_counts()

source     target  value 
Person     gender  Male      14281
                   Female     5867
Character  gender  Male         15
Person     gender  Other         8
Character  gender  Female        4
Other      gender  Female        3
                   Male          1
dtype: int64

In [75]:
df = artists_data.groupby(["type", "gender"], dropna=False).count()
df = df.reset_index()
df = df.drop(columns=["location", "members"])
df = df.rename(columns={"_id": "value"})
df = df.fillna({col: "unknown_"+col for col in df.columns})
df = df.replace({col:{"Other": "Other_"+col for _ in df.columns} for col in df.columns})
df

,type,gender,value
0,Character,Female,4
1,Character,Male,15
2,Character,unknown_gender,6
3,Choir,unknown_gender,44
4,Group,unknown_gender,29806
5,Orchestra,unknown_gender,30
6,Other_type,Female,3
7,Other_type,Male,1
8,Other_type,unknown_gender,17
9,Person,Female,5867


In [76]:
df.columns = ["source", "target", "value"]
df.to_csv("sankey.csv", index=False)

,_id,gender,location,members,type
0,ObjectId(56d7e91b6b60c09814f93e4a),NaN,"{""id_city_musicbrainz"":""6e2d2d30-dbc9-4d27-99f...","[{""id_member_musicbrainz"":""3ec05e94-bf6e-439f-...",Group
1,ObjectId(56d7e91c6b60c09814f93e4c),NaN,"{""id_city_musicbrainz"":"""",""country"":""Japan"",""c...","[{""id_member_musicbrainz"":""82bd3da4-7085-40b8-...",Group
2,ObjectId(56d7e91d6b60c09814f93e4e),NaN,"{""id_city_musicbrainz"":"""",""country"":""Netherlan...","[{""id_member_musicbrainz"":""2931cbb9-56a0-4a96-...",Group
3,ObjectId(56d7e91e6b60c09814f93e50),NaN,"{""id_city_musicbrainz"":"""",""country"":"""",""city"":""""}",[],NaN
4,ObjectId(56d7e91e6b60c09814f93e52),NaN,"{""id_city_musicbrainz"":"""",""country"":""United St...","[{""id_member_musicbrainz"":""2f647f4c-6272-4ad9-...",Group
...,...,...,...,...,...
77487,ObjectId(56d997b1cc2ddd0c0f6bf2d2),NaN,"{""id_city_musicbrainz"":"""",""country"":"""",""city"":""""}",[],Person
77488,ObjectId(56d997b3cc2ddd0c0f6bf2d3),NaN,"{""id_city_musicbrainz"":"""",""country"":""United St...","[{""id_member_musicbrainz"":""1d335878-c0c0-4ded-...",Group
77489,ObjectId(56d997b3cc2ddd0c0f6bf2d4),NaN,"{""id_city_musicbrainz"":"""",""country"":"""",""city"":""""}",[],Group
77490,ObjectId(56d997b3cc2ddd0c0f6bf2d5),NaN,"{""id_city_musicbrainz"":"""",""country"":"""",""city"":""""}",[],Person
